In [1]:
pip install pandas 

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\python38\python.exe -m pip install --upgrade pip' command.


In [2]:
#import config
import random
import json
import nltk
import sklearn

import telebot
import requests
#from telebot import types
from bs4 import BeautifulSoup as BS


#import yfinance as yf
import pandas as pd
#import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import RandomForestClassifier


In [3]:
with open('C:/tmp/BOT_CONFIG.json', 'r', encoding = 'utf8') as f:
    BOT_CONFIG = json.load(f)

In [4]:
corpus = []
y = []
for intent in BOT_CONFIG['intents'].keys():
    for example in BOT_CONFIG['intents'][intent]['examples']:
        corpus.append(example)
        y.append(intent)

In [5]:
corpus_train, corpus_test, y_train, y_test = sklearn.model_selection.train_test_split(corpus, y, test_size=0.2)

In [6]:
vectorizer = sklearn.feature_extraction.text.CountVectorizer(ngram_range=(2,4), analyzer='char_wb')
#vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(ngram_range=(2, 4), analyzer='char_wb', use_idf=True)
X_train = vectorizer.fit_transform(corpus_train)
X_test = vectorizer.transform(corpus_test)
print(len(vectorizer.get_feature_names_out()))

570


In [7]:
#clf = sklearn.linear_model.RidgeClassifier(copy_X=True, max_iter=200)
# clf = sklearn.ensemble.RandomForestClassifier(
#clf.fit(X_test, y_test)

model = RandomForestClassifier(n_estimators=300) # LogisticRegression(max_iter=200)#
model.fit(X_train, y_train) # учим модель на тренировочной части
model.score(X_train, y_train), model.score(X_test, y_test)

(1.0, 0.8947368421052632)

In [8]:
def currency(text):
    data = requests.get('https://www.cbr-xml-daily.ru/daily_json.js').json()
    return (data['Valute'][text]['Value'])

def wheather(city):
    wheather = []
    r = requests.get('https://rp5.ru/'+city)
    html = BS(r.content,'html.parser')

    tmp = html.find('meta', attrs = {"name":'description'})
    tmp = tmp.attrs['content']
    descript = tmp.split('.')
    descript.pop(-1)
    descript = '.'.join(descript) 
    
    
    #for el in html.select('#ftab_6_content'):
    #    wheather.append(el.select('.t_0, .otstup')[1].text)
    #    wheather.append(el.select('.t_0, .otstup')[0].text)
    #    wheather.append(el.select('.wv_0')[1].text)
    #    wheather.append(el.select('.wv_0')[0].text)
    #s = ' '.join(wheather)
    return descript



def clean(text):
  clean_text = ''
  for char in text.lower():
    if char in 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя-!?':
      clean_text = clean_text + char
  return clean_text

def get_intent(text):
  #from nltk.metrics import edit_distance
  for intent in BOT_CONFIG['intents'].keys():
    for example in BOT_CONFIG['intents'][intent]['examples']:
      s1 = clean(text)
      s2 = clean(example)
      if nltk.edit_distance(s1, s2) / max(len(s1), len(s2)) < 0.4:
        return intent
  return 'intent not found :('

def get_intent_by_model(text):
    print (text)
    return model.predict(vectorizer.transform([text]))[0]




def bot2(text):
    #intent = get_intent(text)
    intent = get_intent_by_model(text)
    print (intent)
    if intent == 'USD' or intent == 'EUR':
        answer = (f'Курс {intent} = {currency(intent)}')
        return answer
    
    else:
        answer = random.choice(BOT_CONFIG['intents'][intent]['responses'])
        tmp = intent.split('_')
        if tmp[0] == 'WHEATHER':
            return wheather(answer)
    return answer

In [9]:
bot2('привет!')

привет!
USD


'Курс USD = 56.4783'

In [10]:

bot = telebot.TeleBot('****************************')

@bot.message_handler(content_types=["text"])

def repeat_all_messages(message): # Название функции не играет никакой роли
    mess = message.text
    bot2(mess)
    bot.send_message(message.chat.id, bot2(mess))
    
bot.polling()

Привет
USD
Привет
USD
Как дела
USD
Как дела
USD
Погода клг
WHEATHER_KALD
Погода клг
WHEATHER_KALD
Евро
EUR
Евро
EUR
Бай
USD
Бай
USD


In [11]:
def wheather(city):
    
    wheather_list = []
    r = requests.get('https://rp5.ru/'+city)
    html = BS(r.content,'html.parser')
    wind1 = html.find('meta', attrs = {"name":'description'})
    wheat = wind1.attrs['content']
    tmp = wheat.split('.')
    print (tmp)
    tmp.pop(-1)
    print (tmp)
    tmp = '.'.join(tmp) 
    print (tmp)
    
    print (wheat)
    for el in html.select('#ftab_6_content'):
        #wheather_list.append(el.select('.t_0, .otstup')[1].text)
        #wheather_list.append(el.select('.t_0, .otstup')[0].text)
         
        wind2 = el.select('.title , .content') #="description"
                                   #data_soup.find_all(attrs={"data-foo": "value"}) 
                                   # [<div data-foo="value">foo!</div>]
        #cloud = el.select('.cc_1, .cc_0')[1].text
    
        #wheather_list.append(el.select('.wv_0')[1].text)
        #wheather_list.append(el.select('.wv_0')[0].text)


    #print (wind)
    return print (wind1) #, wind2) #wheather_list


In [12]:
wheather('Погода в Калининграде')

['В Калининграде сегодня ожидается +24', '', '+23 °C, без осадков, легкий ветер', ' Завтра: +26', '', '+23 °C, cлабый дождь, слабый ветер', ' РП5']
['В Калининграде сегодня ожидается +24', '', '+23 °C, без осадков, легкий ветер', ' Завтра: +26', '', '+23 °C, cлабый дождь, слабый ветер']
В Калининграде сегодня ожидается +24..+23 °C, без осадков, легкий ветер. Завтра: +26..+23 °C, cлабый дождь, слабый ветер
В Калининграде сегодня ожидается +24..+23 °C, без осадков, легкий ветер. Завтра: +26..+23 °C, cлабый дождь, слабый ветер. РП5
<meta content="В Калининграде сегодня ожидается +24..+23 °C, без осадков, легкий ветер. Завтра: +26..+23 °C, cлабый дождь, слабый ветер. РП5" name="description"/>


In [13]:
s

NameError: name 's' is not defined

In [ ]:
ss = ' '.join(s)

In [ ]:
ss